In [1]:
import sys
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
import sumolib
import traci
from sumolib import checkBinary
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import sys
import io
from contextlib import redirect_stdout
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
from utils import *


if 'SUMO_HOME' in os.environ:
    print('SUMO_HOME found')
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))

sumoBinary = checkBinary('sumo-gui')
# sumoBinary = checkBinary('sumo')
roadNetwork = "./config/osm.sumocfg"
sumoCmd = [sumoBinary, "-c", roadNetwork, "--start", "--quit-on-end"]
# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

SUMO_HOME found
SUMO_HOME found
Using device: cuda


In [2]:
planned_path = get_planned_path()
checkpoints = list(planned_path.values())
checkpoints = torch.tensor(checkpoints).float() / 10

 Retrying in 1 seconds
***Starting server on port 60865 ***
Loading net-file from './config/osm.net.xml.gz' ... done (111ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.
Simulation ended at time: 10777.00
Reason: TraCI requested termination.
Performance: 
 Duration: 9.01s
 TraCI-Duration: 5.29s
 Real time factor: 1196.25
 UPS: 130009.768010
Vehicles: 
 Inserted: 2199
 Running: 0
 Waiting: 0
Statistics (avg of 2199):
 RouteLength: 4984.74
 Speed: 9.48
 Duration: 532.63
 WaitingTime: 19.92
 TimeLoss: 76.98
 DepartDelay: 0.53



In [3]:
def project_to_nearest(prediction, planned_path):
    with torch.no_grad():
        starts = planned_path[:, :-1, :]
        to = planned_path[:, 1:, :]

        prediction = prediction.unsqueeze(1).repeat(1, starts.shape[1], 1)
        ap = prediction - starts
        ab = to - starts
        numerator = torch.einsum('ijk,ijk->ij', ap, ab)
        denominator = torch.einsum('ijk,ijk->ij', ab, ab)
        t = numerator / denominator
        t = torch.nan_to_num(t, nan=0.0)
        t = torch.clamp(t, 0, 1)
        projections = starts + t.unsqueeze(2) * ab
        diff = projections - prediction
        distances = torch.norm(diff, dim=2)
        min_indices = torch.argmin(distances, dim=1)
        projections = projections[range(projections.shape[0]), min_indices]
        return projections, min_indices


def project_to_nearest_with_checkpoints(prediction, planned_path):
    with torch.no_grad():
        starts = planned_path[:, :-1, :]
        to = planned_path[:, 1:, :]

        prediction = prediction.unsqueeze(1).repeat(1, starts.shape[1], 1)
        ap = prediction - starts
        ab = to - starts
        numerator = torch.einsum('ijk,ijk->ij', ap, ab)
        denominator = torch.einsum('ijk,ijk->ij', ab, ab)
        t = numerator / denominator
        t = torch.nan_to_num(t, nan=0.0)
        t = torch.clamp(t, 0, 1)
        projections = starts + t.unsqueeze(2) * ab
        diff = projections - prediction
        distances = torch.norm(diff, dim=2)
        min_indices = torch.argmin(distances, dim=1)
        projections = projections[range(projections.shape[0]), min_indices]
        next_checkpoint = to[range(to.shape[0]), min_indices]
        pad_to = F.pad(to, (0, 0, 1, 0), value=0)
        next_next_checkpoint = pad_to[range(to.shape[0]), min_indices + 1]
        projections_with_checkpoints = torch.cat((projections, next_checkpoint, next_next_checkpoint), dim=1)
        return projections_with_checkpoints

In [4]:
# Example DataFrame loading
df = pd.read_csv('edinburgh_trajectories.csv')
num_cols = df.shape[1]
position_indices = [i for i in range(num_cols) if i % 4 == 1 or i % 4 == 2]
position_df = df.iloc[:, position_indices]
position_array = position_df.to_numpy()
sequence_length = len(position_indices) // 2
tensor_list = []

for row in position_array:
    reshaped_tensor = torch.tensor(row.reshape(sequence_length, 2))
    tensor_list.append(reshaped_tensor)

all_trajectories_tensor = torch.stack(tensor_list).float() / 10

next_checkpoint = torch.zeros_like(all_trajectories_tensor)
next_next_checkpoint = torch.zeros_like(all_trajectories_tensor)
checkpoints_pad_1 = F.pad(checkpoints, (0, 0, 1, 0))
for i in range(all_trajectories_tensor.shape[1]):
    _, min_indices = project_to_nearest(all_trajectories_tensor[:, i], checkpoints)
    next_checkpoint[:, i] = checkpoints_pad_1[range(checkpoints.shape[0]), min_indices+1]
    next_next_checkpoint[:, i] = checkpoints_pad_1[range(checkpoints.shape[0]), min_indices+2]

all_trajectories_tensor = torch.cat((all_trajectories_tensor, next_checkpoint, next_next_checkpoint), dim=2)

def add_noise(sequence, path, noise_level=500):
    noise = torch.rand(1) * noise_level
    return sequence + noise, path + noise  

def missing(x, y, z, p):
    return torch.tensor(np.repeat(np.random.rand(x * y) < p, z).reshape(x, y, z)).float()

def generate_masks(tensors, min_mask_ratio=0.0, max_mask_ratio=0.1, missing_ratio=0.6, complete_traj_ratio=0.75):
    initial_masks = missing(tensors.shape[0], tensors.shape[1], tensors.shape[2], missing_ratio)
    masks = []
    for initial_mask in initial_masks:
        if np.random.rand() < complete_traj_ratio:
            masks.append(torch.zeros_like(initial_mask).tolist())
            continue
        seq_length = initial_mask.shape[0]
        mask_start = np.random.randint(int(seq_length * min_mask_ratio), int(seq_length * max_mask_ratio))
        mask = torch.zeros_like(initial_mask)
        mask[:, :mask_start] = 1
        mask = initial_mask * mask
        mask[0] = 0
        mask[1] = 0
        masks.append(mask.tolist())
    return torch.tensor(masks)

# split the data into training and validation sets
# because the data is randomly generated, we don't need to shuffle it
train_ratio = 0.8
train_size = int(train_ratio * all_trajectories_tensor.shape[0])
train_trajectories_tensor = all_trajectories_tensor[:train_size]
val_trajectories_tensor = all_trajectories_tensor[train_size:]

train_checkpoints = checkpoints[:train_size]
val_checkpoints = checkpoints[train_size:]

train_mask = generate_masks(train_trajectories_tensor)

class DatasetWithPlans(Dataset):
    def __init__(self, tensor, input_mask, checkpoints):
        self.tensor = tensor.float().to(device)
        self.input_mask = input_mask.float().to(device)
        self.checkpoints = checkpoints.float().to(device)
    def __len__(self):
        return len(self.tensor)
    def __getitem__(self, idx):
        return self.tensor[idx], self.input_mask[idx], self.checkpoints[idx]
    
train_dataset = DatasetWithPlans(train_trajectories_tensor, train_mask, train_checkpoints)

In [5]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, int(hidden_size/2))
        self.fc2 = nn.Linear(int(hidden_size/2), num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x, h0=None):
        if h0 is None:
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, hidden = self.gru(x, h0)  
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        out = self.fc2(out)
        # out = self.relu(out)
        return out, hidden

class CustomMSE(nn.Module):
    def __init__(self):
        super(CustomMSE, self).__init__()

    def forward(self, output, target):
        # where target is 0
        mask = (target != 0).float().to(device)
        mse = torch.mean(((output - target) ** 2) * mask)
        directional_diff = torch.mean(torch.abs(torch.atan2(output[:, 1], output[:, 0]) - torch.atan2(target[:, 1], target[:, 0])))
        distance_diff = torch.mean(torch.norm(output, dim=1)-torch.norm(target, dim=1))
        return mse + directional_diff + distance_diff

In [6]:
def train_model(model, dataloader, epochs, optimizer, criterion):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, masks, planned_path in dataloader:
            optimizer.zero_grad()
            hidden = None
            loss = 0
            # Autoregressive prediction
            # Start with the first input and predict each subsequent step
            seq_len = inputs.size(1)
            current_input = inputs[:, 0, :].unsqueeze(1)
            # steps = 0
            for t in range(1, seq_len):
                # new_steps = int(sum((inputs[:, t, :2].reshape(-1) != 0).float().to(device))) / 2
                # if new_steps == 0:
                #     break
                prediction, hidden = model(current_input, hidden)
                # steps += new_steps
                previous_input = inputs[:, t-1, :2]
                if epoch > 25:
                    projection_with_checkpoints = project_to_nearest_with_checkpoints(prediction, planned_path)
                    current_input = (projection_with_checkpoints * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
                else:
                    current_input = inputs[:, t, :].unsqueeze(1)
                if t > 1:
                    loss += criterion(prediction - previous_input, inputs[:, t, :2]-previous_input) #(current_input-previous_input).squeeze(1))
            # loss /= steps
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
 
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

model = GRUModel(input_size=6, hidden_size=256, num_layers=2, num_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CustomMSE()
train_model(model, train_dataloader, 60, optimizer, criterion)

Epoch 1, Loss: 73959513.28571428
Epoch 2, Loss: 59192906.14285714
Epoch 3, Loss: 35430819.0
Epoch 4, Loss: 16629504.714285715
Epoch 5, Loss: 11095387.142857144
Epoch 6, Loss: 7278351.75
Epoch 7, Loss: 5813318.928571428
Epoch 8, Loss: 4803673.866071428
Epoch 9, Loss: 3334762.223214286
Epoch 10, Loss: 1874984.9375
Epoch 11, Loss: 902971.3883928572
Epoch 12, Loss: 469575.984375
Epoch 13, Loss: 367703.13727678574
Epoch 14, Loss: 250519.98158482142
Epoch 15, Loss: 224777.7779017857
Epoch 16, Loss: 200616.37667410713
Epoch 17, Loss: 197462.35770089287
Epoch 18, Loss: 183480.77957589287
Epoch 19, Loss: 237478.1830357143
Epoch 20, Loss: 184041.39676339287
Epoch 21, Loss: 173763.5122767857
Epoch 22, Loss: 170935.875
Epoch 23, Loss: 170129.04017857142
Epoch 24, Loss: 178499.12220982142
Epoch 25, Loss: 174839.79799107142
Epoch 26, Loss: 172034.5005580357
Epoch 27, Loss: 209797.3549107143
Epoch 28, Loss: 200894.60379464287
Epoch 29, Loss: 202167.49497767858
Epoch 30, Loss: 187007.16573660713
Epoch

In [10]:
def checkModel(model, inputs, masks, paths):
    model.eval()
    with torch.no_grad():
        hidden = None
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            prediction, hidden = model(current_input, hidden)
            projection_with_checkpoints = project_to_nearest_with_checkpoints(prediction, paths)
            current_input = (projection_with_checkpoints * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            print(projection_with_checkpoints[0][:2]*10, inputs[:, t, :][0][:2]*10)

# get the first batch in dataloader
val_mask = generate_masks(val_trajectories_tensor, missing_ratio=0.9, complete_traj_ratio=0)
val_dataset = DatasetWithPlans(val_trajectories_tensor, val_mask, val_checkpoints)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
inputs, masks, paths = next(iter(val_dataloader))

checkModel(model, inputs, masks, paths)

tensor([4116.6401, 2838.5400], device='cuda:0') tensor([5324.8594,   46.8790], device='cuda:0')
tensor([5414.9561,  326.0851], device='cuda:0') tensor([5326.2979,   50.8659], device='cuda:0')
tensor([5440.0874,  405.9027], device='cuda:0') tensor([5328.2666,   56.8385], device='cuda:0')
tensor([5444.3193,  419.2924], device='cuda:0') tensor([5330.6602,   64.1015], device='cuda:0')
tensor([5444.8560,  420.9892], device='cuda:0') tensor([5333.3472,   72.2500], device='cuda:0')
tensor([5444.0532,  418.4508], device='cuda:0') tensor([5336.2305,   80.9964], device='cuda:0')
tensor([5442.6567,  414.0331], device='cuda:0') tensor([5338.8389,   88.9111], device='cuda:0')
tensor([5441.1187,  409.1663], device='cuda:0') tensor([5341.5732,   97.1891], device='cuda:0')
tensor([5439.6123,  404.4002], device='cuda:0') tensor([5344.4600,  105.8772], device='cuda:0')
tensor([5438.2422,  400.0642], device='cuda:0') tensor([5347.3037,  114.4378], device='cuda:0')
tensor([5437.0796,  396.3851], device='c

In [11]:
def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0
    steps = 0
    for inputs, masks, paths in dataloader:
        hidden = None
        loss = 0
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            new_steps = int(sum((inputs[:, t, :2].reshape(-1) != 0).float().to(device))) / 2
            if new_steps == 0:
                break
            steps += new_steps
            prediction, hidden = model(current_input, hidden)
            projection_with_checkpoints = project_to_nearest_with_checkpoints(prediction, paths)
            current_input = (projection_with_checkpoints * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            loss += torch.norm((projection_with_checkpoints[:, :2] - inputs[:, t, :2]) * 10, dim=1) * (inputs[:, t, :2] != 0)[:, 0]
        total_loss += loss.sum().item()
    print(f'Validation Loss: {total_loss / steps}')

# model = GRUModel(input_size=6, hidden_size=128, num_layers=2, num_classes=2).to(device)
# model.load_state_dict(torch.load('gru_model_e.pth'))
for i in [0.9, 0.8, 0.7, 0.6, 0.5, 0]:
    val_mask = generate_masks(val_trajectories_tensor, missing_ratio=i, complete_traj_ratio=0)
    val_dataset = DatasetWithPlans(val_trajectories_tensor, val_mask, val_checkpoints)
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
    evaluate_model(model, val_dataloader)

Validation Loss: 291.75596782786533
Validation Loss: 177.20961726846204
Validation Loss: 131.65232139481327
Validation Loss: 102.74263151563063
Validation Loss: 85.94473314486054
Validation Loss: 47.36378991441387
